# CNN으로 MNIST 정확도를 높여보자

In [1]:
# 이미지 처리 분야에서 가장 유명한 신경망 모델인 CNN 을 이용하여 더 높은 인식률을 만들어봅니다.
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


# 신경망 모델 구성

In [2]:
# 기존 모델에서는 입력 값을 28x28 하나의 차원으로 구성하였으나,
# CNN 모델을 사용하기 위해 2차원 평면과 특성치의 형태를 갖는 구조로 만듭니다.
# 1은 특성수를 의미.
X = tf.placeholder(tf.float32, [None, 28, 28, 1])  # None은 몇개를 받을지 아직 모른다는 의미.
Y = tf.placeholder(tf.float32, [None, 10])         # 0부터 9까지 중 판단하는 것이므로 10
keep_prob = tf.placeholder(tf.float32)

In [3]:
# 각각의 변수와 레이어는 다음과 같은 형태로 구성됩니다.
# W1 [3 3 1 32] -> [3 3]: 필터 크기, 1: 입력값 X 의 특성수, 32: 필터 갯수
# L1 Conv shape=(?, 28, 28, 32)
#    Pool     ->(?, 14, 14, 32)
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
# tf.nn.conv2d 를 이용해 한칸씩 움직이는 컨볼루션 레이어를 쉽게 만들 수 있습니다.
# padding='SAME' 은 커널 슬라이딩시 최외곽에서 한칸 밖으로 더 움직이는 옵션
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
print("L1 Conv shape =",L1.shape)
# Pooling 역시 tf.nn.max_pool 을 이용하여 쉽게 구성할 수 있습니다.
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
print("L1 Pool shape =", L1.shape)
# L1 = tf.nn.dropout(L1, keep_prob)

L1 Conv shape = (?, 28, 28, 32)
L1 Pool shape = (?, 14, 14, 32)


In [4]:
# L2 Conv shape=(?, 14, 14, 64)
#    Pool     ->(?, 7, 7, 64)
# W2 의 [3, 3, 32, 64] 에서 32 는 L1 에서 출력된 W1 의 마지막 차원, 필터의 크기 입니다.
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))  #64는 필터의 갯수.
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
print("L2 Conv shape =",L2.shape)
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
print("L2 Pool shape =",L2.shape)
# L2 = tf.nn.dropout(L2, keep_prob)

L2 Conv shape = (?, 14, 14, 64)
L2 Pool shape = (?, 7, 7, 64)


In [5]:
# FC 레이어: 입력값 7x7x64 -> 출력값 256
# Full connect를 위해 직전의 Pool 사이즈인 (?, 7, 7, 64) 를 참고하여 차원을 줄여줍니다.
#    Reshape  ->(?, 256)
W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7 * 7 * 64])   # L2 를 재구성
print("L3(L2 reshape) size =",L3.shape )
L3 = tf.matmul(L3, W3)
print("L3 shape after tf.matmul(L3, W3) =",L3.shape)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob)

L3(L2 reshape) size = (?, 3136)
L3 shape after tf.matmul(L3, W3) = (?, 256)


In [6]:
# 최종 출력값 L3 에서의 출력 256개를 입력값으로 받아서 0~9 레이블인 10개의 출력값을 만듭니다.
W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)
print("Model shape after tf.matmul(L3, W4) =", model.shape)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)
# 최적화 함수를 RMSPropOptimizer 로 바꿔서 결과를 확인해봅시다.
# optimizer = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)

Model shape after tf.matmul(L3, W4) = (?, 10)


# 신경망 모델 학습

In [7]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(15):
    total_cost = 0

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # 이미지 데이터를 CNN 모델을 위한 자료형태인 [28 28 1] 의 형태로 재구성합니다.
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)

        _, cost_val = sess.run([optimizer, cost],
                               feed_dict={X: batch_xs,
                                          Y: batch_ys,
                                          keep_prob: 0.7})
        total_cost += cost_val

    print('Epoch:', '%04d' % (epoch + 1),
          'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))

print('최적화 완료!')

Epoch: 0001 Avg. cost = 0.389
Epoch: 0002 Avg. cost = 0.116
Epoch: 0003 Avg. cost = 0.084
Epoch: 0004 Avg. cost = 0.067
Epoch: 0005 Avg. cost = 0.053
Epoch: 0006 Avg. cost = 0.046
Epoch: 0007 Avg. cost = 0.039
Epoch: 0008 Avg. cost = 0.033
Epoch: 0009 Avg. cost = 0.031
Epoch: 0010 Avg. cost = 0.027
Epoch: 0011 Avg. cost = 0.023
Epoch: 0012 Avg. cost = 0.022
Epoch: 0013 Avg. cost = 0.020
Epoch: 0014 Avg. cost = 0.017
Epoch: 0015 Avg. cost = 0.016
최적화 완료!
